# Experimentation Only

In [1]:
import sys
!{sys.executable} -m pip install tqdm

You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#For Prediction
from sklearn import metrics
from sklearn import preprocessing, cross_validation
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Dense, Dropout, Input, LSTM, Activation, Flatten
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Read Datasets 
research_data_raw = pd.read_csv('Datasets/bitcoin_research_data_v2.csv')
# Drop unnecessary columns
research_data = research_data_raw.drop('Unnamed: 0', axis=1)
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0


In [4]:
# Convert data to milliseconds
research_data['Epoch'] = pd.to_datetime(research_data['Date'] + ' 23:59:59.000')
research_data['Epoch'] = (research_data['Epoch'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599


In [5]:
research_data_imp2 = research_data.copy()
research_data_imp2.sort_values(by='Date', inplace=True, ascending=True) # Sort by date ascending
forecast_out = int(math.ceil(0.0001 * len(research_data_imp2))) # 0.01% of data
#forecast_out = int(math.ceil(0.0025 * len(research_data_imp2))) # 0.05% of data
print(len(research_data_imp2))
print(forecast_out)

1808
1


In [6]:
research_data_imp2['Prediction'] = research_data_imp2['Close'].shift(-forecast_out)
research_data_imp2.tail()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch,Prediction
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599,5939.6
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999,6151.4
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399,6337.9
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799,6793.5
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199,NaN


In [7]:
def load_data():
    X = np.array(research_data_imp2.drop(['Prediction', 'Symbol', 'Date', 'Popularity'], 1)) # Create X with numeric columns only
    # X = np.array(research_data_imp2['Close'])
    # X = research_data_imp2['Close'].to_frame()
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:]
    X = X[:-forecast_out]

    # research_data_imp2.dropna(inplace=True)

    y = np.array(research_data_imp2['Prediction'])
    y = y[:-forecast_out]

    ##X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

    # split data into training set and a temporary set using sklearn.model_selection.traing_test_split
    #X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.2, random_state=23)
    
    # take the remaining 20% of data in X_tmp, y_tmp and split them evenly
    #X_test, X_val, y_test, y_val = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=23)
    
    #return [X_train, X_test, X_val, y_train, y_test, y_val]
    
    train_size = int(len(X) * .75)
    X_train, X_test = X[0:train_size], X[train_size:len(X)]
    y_train, y_test = y[0:train_size], y[train_size:len(y)]
    
    return [X_train, X_test, y_train, y_test]
    
    

In [8]:
def load_data_with_3D_array():
    #input_cols = ['Open', 'High', 'Low', 'Close', 'Volume BTC', 'Volume USD', 'Popularity_Normalized', 'Epoch']
    #input_cols = ['Close', 'Popularity_Normalized']
    input_cols = ['Close']
    
    df = research_data_imp2.copy()
    #df.dropna(inplace=True)
    
    # Put your inputs into a single list
    df['single_input_vector'] = df[input_cols].apply(tuple, axis=1).apply(list)
    
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
    
    # Use .cumsum() to include previous row vectors in the current row list of vectors
    df['cumulative_input_vectors'] = df.single_input_vector.cumsum()
    
    # If your output is multi-dimensional, you need to capture those dimensions in one object
    # If your output is a single dimension, this step may be unnecessary
    # df['output_vector'] = df[output_cols].apply(tuple, axis=1).apply(list)
    
    # Pad your sequences so they are the same length
    max_sequence_length = df.cumulative_input_vectors.apply(len).max()
    
    # Save it as a list   
    padded_sequences = pad_sequences(df.cumulative_input_vectors.tolist(), max_sequence_length, dtype='float32').tolist()
    df['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)
    
    # print(df)
    
    # Extract your training data
    X_train_init = np.asarray(df.padded_input_vectors)
    
    # Use hstack to and reshape to make the inputs a 3d vector
    X = np.hstack(X_train_init).reshape(len(df),max_sequence_length,len(input_cols))
    #y_train = np.hstack(np.asarray(df.output_vector)).reshape(len(df),len(output_cols))
    y = np.hstack(np.asarray(df.Prediction)).reshape(len(df),1)
    
    return [df, X, y]
    
    #train_size = int(len(X) * .75)
    #X_train, X_test = X[0:train_size], X[train_size:len(X)]
    #y_train, y_test = y[0:train_size], y[train_size:len(y)]
    
    #return [X_train, X_test, y_train, y_test]

In [9]:
# (X_train, X_test, X_val, y_train, y_test, y_val) = load_data()
# print(X_train.shape)
# print(X_test.shape)
# print(X_val.shape)
# print(y_train.shape)
# print(y_test.shape)
# print(y_val.shape)

# (X_train, X_test,y_train, y_test) = load_data()

#(X_train, X_test,y_train, y_test) = load_data_with_3D_array()

#research_data_imp2.drop(research_data_imp2.tail(1).index,inplace=True)
#research_data_imp2.iloc[1807]['Prediction']
research_data_imp2.set_value(0, 'Prediction', 6793.5)
(df, X_train, y_train) = load_data_with_3D_array()
df

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch,Prediction,single_input_vector,cumulative_input_vectors,padded_input_vectors
1807,2014-05-18,BTCUSD,453.83,453.83,445.56,445.56,2.580,1151.52,2.0,200.0,1400457599,445.56,[[445.56]],[[445.56]],"[[445.55999755859375], [445.55999755859375], [..."
1806,2014-05-19,BTCUSD,445.56,445.56,445.56,445.56,0.000,0.00,2.0,200.0,1400543999,493.43,[[445.56]],"[[445.56], [445.56]]","[[0.0], [445.55999755859375], [445.55999755859..."
1805,2014-05-20,BTCUSD,445.56,498.84,445.56,493.43,25.280,11926.23,2.0,200.0,1400630399,494.77,[[493.43]],"[[445.56], [445.56], [493.43]]","[[0.0], [0.0], [445.55999755859375], [445.5599..."
1804,2014-05-21,BTCUSD,493.43,497.67,490.85,494.77,4.240,2092.04,2.0,200.0,1400716799,530.00,[[494.77]],"[[445.56], [445.56], [493.43], [494.77]]","[[0.0], [0.0], [0.0], [445.55999755859375], [4..."
1803,2014-05-22,BTCUSD,494.77,530.00,494.77,530.00,32.850,16875.67,2.0,200.0,1400803199,521.47,[[530.0]],"[[445.56], [445.56], [493.43], [494.77], [530.0]]","[[0.0], [0.0], [0.0], [0.0], [445.559997558593..."
1802,2014-05-23,BTCUSD,530.00,548.05,515.31,521.47,14.820,7949.09,2.0,200.0,1400889599,530.00,[[521.47]],"[[445.56], [445.56], [493.43], [494.77], [530....","[[0.0], [0.0], [0.0], [0.0], [0.0], [445.55999..."
1801,2014-05-24,BTCUSD,521.47,530.00,515.21,530.00,2.620,1364.13,2.0,200.0,1400975999,574.05,[[530.0]],"[[445.56], [445.56], [493.43], [494.77], [530....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [44..."
1800,2014-05-25,BTCUSD,530.00,589.00,530.00,574.05,15.830,8890.61,3.0,300.0,1401062399,583.20,[[574.05]],"[[445.56], [445.56], [493.43], [494.77], [530....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
1799,2014-05-26,BTCUSD,574.05,589.11,563.68,583.20,10.950,6369.72,3.0,300.0,1401148799,570.92,[[583.2]],"[[445.56], [445.56], [493.43], [494.77], [530....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
1798,2014-05-27,BTCUSD,583.20,594.64,538.84,570.92,26.920,15396.82,3.0,300.0,1401235199,575.83,[[570.92]],"[[445.56], [445.56], [493.43], [494.77], [530....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."


In [12]:
def create_lstm():
    model = Sequential()
    
#     model.add(LSTM(32, input_shape=(1, 8), return_sequences=True))
#     #model.add(Dropout(0.4))
#     #model.add(LSTM(16, return_sequences=False))
#     model.add(Dense(1))
#     model.add(Activation('tanh'))
#     optimizer = RMSprop(lr=0.005, clipvalue=1.)
    
    #model.add(LSTM(512, input_shape=(1, 8), dropout_U = 0.2, dropout_W = 0.2, return_sequences=True))
    #model.add(LSTM(256, return_sequences=True))
    #model.add(Dropout(0.5))
    #model.add(LSTM(128, return_sequences=True))
    #model.add(LSTM(64))
    #model.add(Dropout(0.5))
    
    #model.add(LSTM(512, input_shape=(1808, 1), dropout_U = 0.2, dropout_W = 0.2, return_sequences=True))
    #model.add(LSTM(256, return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(LSTM(512, input_shape=(1808, 1), dropout_U = 0.2, dropout_W = 0.2, return_sequences=True))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = RMSprop(lr=0.0001, decay=3e-8)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

lstm = create_lstm()
lstm.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, input_shape=(1808, 1), return_sequences=True, recurrent_dropout=0.2, dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1808, 512)         1052672   
_________________________________________________________________
lstm_8 (LSTM)                (None, 1808, 64)          147712    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1808, 64)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,212,833
Trainable params: 1,212,833
Non-trainable params: 0
_________________________________________________________________


In [13]:
#X_train2 = np.reshape(X_train, (X_train.shape[0], X_train.shape[0] , X_train.shape[1])) #X.reshape(samples, timesteps, features)
#X_test2 = np.reshape(X_test, (X_test.shape[0], X_test.shape[0], X_test.shape[1]))
#X_val2 = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

#print(X_train2.shape)
#print(X_test2.shape)
#print(X_val2.shape)

# X_train2 = np.transpose(X_train)
# X_train2 = np.reshape(X_train2, (X_train2.shape[0], X_train2.shape[1], 1)) #X.reshape(samples, timesteps, features)
# print(X_train2.shape)

# X_test2 = np.transpose(X_test)
# X_test2 = np.reshape(X_test2, (X_test2.shape[0], X_test2.shape[1], 1)) #X.reshape(samples, timesteps, features)
# print(X_test2.shape)
# print(y_train.shape)


In [ ]:
#lstm.fit(X_train2, y_train, batch_size=128, epochs=50, validation_data=[X_val2, y_val])

lstm.fit(X_train, y_train, validation_split=0.25, epochs=5, batch_size=32)

Train on 1356 samples, validate on 452 samples
Epoch 1/5


In [12]:
print(X_train.shape)
print(y_train.shape)

(1808, 1808, 1)
(1808, 1)


In [13]:
print(X_train)

[[[ 445.55999756]
  [   0.        ]
  [   0.        ]
  ...
  [   0.        ]
  [   0.        ]
  [   0.        ]]

 [[ 445.55999756]
  [ 445.55999756]
  [   0.        ]
  ...
  [   0.        ]
  [   0.        ]
  [   0.        ]]

 [[ 493.42999268]
  [ 445.55999756]
  [ 445.55999756]
  ...
  [   0.        ]
  [   0.        ]
  [   0.        ]]

 ...

 [[6151.39990234]
  [5939.60009766]
  [5906.70019531]
  ...
  [ 445.55999756]
  [   0.        ]
  [   0.        ]]

 [[6337.89990234]
  [6151.39990234]
  [5939.60009766]
  ...
  [ 445.55999756]
  [ 445.55999756]
  [   0.        ]]

 [[6793.5       ]
  [6337.89990234]
  [6151.39990234]
  ...
  [ 493.42999268]
  [ 445.55999756]
  [ 445.55999756]]]


In [14]:
print(y_train)

[[ 445.56]
 [ 493.43]
 [ 494.77]
 ...
 [6337.9 ]
 [6793.5 ]
 [6793.5 ]]
